<a href="https://colab.research.google.com/github/Tanvir-Shakil-Joy/CNN/blob/main/Crop_Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collecting Data

### Importing dataset from kaggle

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle datasets download -d nafishamoin/bangladeshi-crops-disease-dataset

In [ ]:
! unzip bangladeshi-crops-disease-dataset.zip

### Splitting the dataset into train and test

In [ ]:
! pip install split-folders


In [ ]:
! mkdir dataset

In [ ]:
import splitfolders
splitfolders.ratio('/content/CropDisease/Crop___DIsease', output="dataset", 
                   seed=42, ratio=(.7, .2, .1), 
                   group_prefix=None)

# Convolutional Neural Network

### Importing the libraries

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

## Part 1 - Data Preprocessing

### Preprocessing the training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

### Preprocessing the test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/dataset/val',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

### Adding class names

In [ ]:
class_names = ['Corn__Common_Rust', 'Corn__Gray_Leaf_Spot', 'Corn__Healthy', 'Corn__Leaf_Blight',
               'Invalid', 'Potato__Early_Blight', 'Potato__Healthy', 'Potato__Late_Blight', 'Rice__Brown_Spot',
               'Rice__Healthy', 'Rice__Hispa', 'Rice__Leaf_Blast', 'Wheat__Brown_Rust', 'Wheat_Healthy',
               'Wheat__Yellow_Rust']

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))

### Step 5 - Output layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=15, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the training set and evaluating it on the test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 50)

### Saving the model

In [ ]:
cnn.save('Crop_Disease_Prediction_CNN_Model.h5')

## Part 4 - Making a Prediction

### Loading the model

In [ ]:
cnn = tf.keras.models.load_model('Crop_Disease_Prediction_CNN_Model.h5')

### Making the prediction

In [ ]:
data = input('Enter the name of the image : ')
img = cv.imread(data)
img = cv.resize(img, [64, 64])
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img, cmap=plt.cm.binary)
prediction = cnn.predict(np.array([img]) / 255)
index = np.argmax(prediction)
print(f'Prediction is {class_names[index]}')